# Hello NPU

## Working with NPU in OpenVINO™

#### Table of contents:
- [Introduction](#Introduction)
    - [Install required packages](#Install-required-packages)
- [Checking NPU with Query Device](#Checking-NPU-with-Query-Device)
    - [List the NPU with core.available_devices](#List-the-NPU-with-core.available_devices)
    - [Check Properties with core.get_property](#Check-Properties-with-core.get_property)
    - [Brief Descriptions of Key Properties](#Brief-Descriptions-of-Key-Properties)
- [Compiling a Model on NPU](#Compiling-a-Model-on-NPU)
    - [Download and Convert a Model](#Download-and-Convert-a-Model)
        - [Download the Model](#Download-the-Model)
        - [Convert the Model to OpenVINO IR format](#Convert-the-Model-to-OpenVINO-IR-format)
    - [Compile with Default Configuration](#Compile-with-Default-Configuration)
    - [Reduce Compile Time through Model Caching](#Reduce-Compile-Time-through-Model-Caching)
        - [UMD Model Caching](#UMD-Model-Caching)
        - [OpenVINO Model Caching](#OpenVINO-Model-Caching)
    - [Throughput and Latency Performance Hints](#Throughput-and-Latency-Performance-Hints)
- [Performance Comparison with benchmark_app](#Performance-Comparison-with-benchmark_app)
    - [NPU vs CPU with Latency Hint](#NPU-vs-CPU-with-Latency-Hint)
        - [Effects of UMD Model Caching](#Effects-of-UMD-Model-Caching)
    - [NPU vs CPU with Throughput Hint](#NPU-vs-CPU-with-Throughput-Hint)
- [Limitations](#Limitations)
- [Conclusion](#Conclusion)

This tutorial provides a high-level overview of working with the NPU device **Intel(R) AI Boost** (introduced with the Intel® Core™ Ultra generation of CPUs) in OpenVINO. It explains some of the key properties of the NPU and shows how to compile a model on NPU with performance hints.

This tutorial also shows example commands for benchmark_app that can be run to compare NPU performance with CPU in different configurations.

## Introduction
[back to top ⬆️](#Table-of-contents:)

The Neural Processing Unit (NPU) is a low power hardware solution which enables you to offload certain neural network computation tasks from other devices, for more streamlined resource management.

Note that the NPU plugin is currently available only with the Archive distribution and PIP installation of OpenVINO™ and you need to [install a proper NPU driver](https://docs.openvino.ai/2024/get-started/configurations/configurations-intel-npu.html) to use it successfully.

**Supported Platforms**:  
    Host: Intel® Core™ Ultra  
    NPU device: NPU 3720  
    OS: Ubuntu 22.04 (with Linux Kernel 6.6+), MS Windows 11 (both 64-bit)  
    

### Install required packages
[back to top ⬆️](#Table-of-contents:)

In [1]:
%pip install -q "openvino-dev>=2024.1.0"
%pip install -q tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Checking NPU with Query Device
[back to top ⬆️](#Table-of-contents:)

In this section, we will see how to list the available NPU and check its properties. Some of the key properties will be defined.

### List the NPU with core.available_devices
[back to top ⬆️](#Table-of-contents:)

OpenVINO Runtime provides the ```available_devices``` method for checking which devices are available for inference. The following code will output a list a compatible OpenVINO devices, in which Intel NPU should appear (ensure that the driver is installed successfully). 

In [4]:
import openvino as ov

core = ov.Core()
core.available_devices

['CPU', 'GPU', 'NPU']

### Check Properties with core.get_property
[back to top ⬆️](#Table-of-contents:)

To get information about the NPU, we can use device properties. In OpenVINO, devices have properties that describe their characteristics and configurations. Each property has a name and associated value that can be queried with the ```get_property``` method.

To get the value of a property, such as the device name, we can use the ```get_property``` method as follows:

In [5]:
device = "NPU"

core.get_property(device, "FULL_DEVICE_NAME")

'Intel(R) AI Boost'

Each device also has a specific property called ```SUPPORTED_PROPERTIES```, that enables viewing all the available properties in the device. We can check the value for each property by simply looping through the dictionary returned by ```core.get_property("NPU", "SUPPORTED_PROPERTIES")``` and then querying for that property.

In [ ]:
print(f"{device} SUPPORTED_PROPERTIES:\n")
supported_properties = core.get_property(device, "SUPPORTED_PROPERTIES")
indent = len(max(supported_properties, key=len))

for property_key in supported_properties:
    if property_key not in ('SUPPORTED_METRICS', 'SUPPORTED_CONFIG_KEYS', 'SUPPORTED_PROPERTIES'):
        try:
            property_val = core.get_property(device, property_key)
        except TypeError:
            property_val = 'UNSUPPORTED TYPE'
        print(f"{property_key:<{indent}}: {property_val}")

### Brief Descriptions of Key Properties
[back to top ⬆️](#Table-of-contents:)

Each device has several properties as seen in the last command. Some of the key properties are:
- `FULL_DEVICE_NAME` - The product name of the NPU.
- `PERFORMANCE_HINT` - A high-level way to tune the device for a specific performance metric, such as latency or throughput, without worrying about device-specific settings.
- `CACHE_DIR` - The directory where the OpenVINO model cache data is stored to speed up the compilation time.
- `OPTIMIZATION_CAPABILITIES` - The model data types (INT8, FP16, FP32, etc) that are supported by this NPU.

To learn more about devices and properties, see the [Query Device Properties](https://docs.openvino.ai/2024/openvino-workflow/running-inference/inference-devices-and-modes/query-device-properties.html) page.

## Compiling a Model on NPU
[back to top ⬆️](#Table-of-contents:)

Now, we know the NPU present in the system and we have checked its properties. We can easily use it for compiling and running models with OpenVINO NPU plugin (currently only available with Archive and PIP distribution of OpenVINO).

### Download and Convert a Model
[back to top ⬆️](#Table-of-contents:)

This tutorial uses the ```resnet50``` model. The ```resnet50``` model is used for image classification tasks. The model was trained on [ImageNet](https://www.image-net.org/index.php) dataset which contains over a million images categorized into 1000 classes. For details see the [paper](https://ieeexplore.ieee.org/document/7780459).

#### Download the Model
[back to top ⬆️](#Table-of-contents:)

Fetch [ResNet50 CV](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights) Classification model from torchvision.

In [7]:
from pathlib import Path

# create a directory for resnet model file
MODEL_DIRECTORY_PATH = Path("model")
MODEL_DIRECTORY_PATH.mkdir(exist_ok=True)

model_name = "resnet50"

In [8]:
from torchvision.models import resnet50, ResNet50_Weights

# create model object
pytorch_model = resnet50(weights=ResNet50_Weights.DEFAULT)
# switch model from training to inference mode
pytorch_model.eval();

#### Convert the Model to OpenVINO IR format
[back to top ⬆️](#Table-of-contents:)

Convert Pytorch model to ONNX format:

In [9]:
import torch
import warnings

ONNX_CV_MODEL_PATH = MODEL_DIRECTORY_PATH / f'{model_name}.onnx'

if ONNX_CV_MODEL_PATH.exists():
    print(f"ONNX model {ONNX_CV_MODEL_PATH} already exists.")
else:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        torch.onnx.export(
            model=pytorch_model, args=torch.randn(1, 3, 224, 224), f=ONNX_CV_MODEL_PATH
        )
    print(f"ONNX model exported to {ONNX_CV_MODEL_PATH}")

ONNX model model\resnet50.onnx already exists.


To convert this ONNX model to OpenVINO IR with `FP16` precision, use model conversion API. The models are saved to the `model/ir_model/` directory. For more details about model conversion, see this [page](https://docs.openvino.ai/2024/openvino-workflow/model-preparation.html).

In [10]:
precision = 'FP16'

model_path = MODEL_DIRECTORY_PATH / 'ir_model' / f'{model_name}_{precision.lower()}.xml'

model = None
if not model_path.exists():
    model = ov.convert_model(ONNX_CV_MODEL_PATH)
    ov.save_model(model, model_path, compress_to_fp16=(precision == "FP16"))
    print("IR model saved to {}".format(model_path))
else:
    print("Read IR model from {}".format(model_path))
    model = core.read_model(model_path)

Read IR model from model\ir_model\resnet50_fp16.xml


### Compile with Default Configuration
[back to top ⬆️](#Table-of-contents:)

When the model is ready, first we need to read it, using the `read_model` method. Then, we can use the `compile_model` method and specify the name of the device we want to compile the model on, in this case, "NPU".

In [11]:
compiled_model = core.compile_model(model, device)

Any of the device names returned by the `available_devices` method are valid device specifiers.  
You may also use "AUTO", which will automatically select the best device for inference.  
(**Note that NPU is currently excluded from the default priority list. To use it for inference, you need to specify it explicitly**).  
To learn more about AUTO plugin, visit the [Automatic Device Selection](https://docs.openvino.ai/2024/openvino-workflow/running-inference/inference-devices-and-modes/auto-device-selection.html) page.

### Reduce Compile Time through Model Caching
[back to top ⬆️](#Table-of-contents:)

Depending on the model used, device-specific optimizations and network compilations can cause the compile step to be time-consuming, especially with larger models, which may lead to bad user experience in the application. To solve this **Model Caching** can be used.

Model Caching helps reduce application startup delays by exporting and reusing the compiled model automatically. The following two compilation-related metrics are crucial in this area:

- **First-Ever Inference Latency (FEIL)**:  
  Measures all steps  required to compile and execute a model on the device for the first time. It includes model compilation time, the time required to load and initialize the model on the device and the first inference execution.
- **First Inference Latency (FIL)**:  
  Measures the time required to load and initialize the pre-compiled model on the device and the first inference execution.

In NPU, UMD model caching is a solution enabled by default by the driver. It improves time to first inference (FIL) by storing the model in the cache after compilation (included in FEIL). Learn more about UMD Caching [here](https://docs.openvino.ai/2024/openvino-workflow/running-inference/inference-devices-and-modes/npu-device.html#umd-dynamic-model-caching). Due to this caching, it takes lesser time to load the model after first compilation.

You can also use OpenVINO Model Caching, which is a common mechanism for all OpenVINO device plugins and can be enabled by setting the `cache_dir` property.  
By enabling OpenVINO Model Caching, the UMD caching is automatically bypassed by the NPU plugin, which means the model will only be stored in the OpenVINO cache after compilation. When a cache hit occurs for subsequent compilation requests, the plugin will import the model instead of recompiling it.

#### UMD Model Caching
[back to top ⬆️](#Table-of-contents:)

To see how UMD caching see the following example:

In [12]:
import time
from pathlib import Path

start = time.time()
core = ov.Core()

# Compile the model as before
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"UMD Caching (first time) - compile time: {time.time() - start}s")

UMD Caching (first time) - compile time: 2.810673952102661s


In [13]:
start = time.time()
core = ov.Core()

# Compile the model once again to see UMD Caching
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"UMD Caching - compile time: {time.time() - start}s")

UMD Caching - compile time: 2.76252818107605s


#### OpenVINO Model Caching
[back to top ⬆️](#Table-of-contents:)

To get an idea of OpenVINO model caching, we can use the OpenVINO cache as follow

In [14]:
# Create cache folder
cache_folder = Path("cache")
cache_folder.mkdir(exist_ok=True)

start = time.time()
core = ov.Core()

# Set cache folder
core.set_property({'CACHE_DIR': cache_folder})

# Compile the model
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"Cache enabled (first time) - compile time: {time.time() - start}s")

start = time.time()
core = ov.Core()

# Set cache folder
core.set_property({'CACHE_DIR': cache_folder})

# Compile the model as before
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"Cache enabled (second time) - compile time: {time.time() - start}s")

Cache enabled (first time) - compile time: 2.4384639263153076s
Cache enabled (second time) - compile time: 0.3303408622741699s


And when the OpenVINO cache is disabled:

In [15]:
start = time.time()
core = ov.Core()
model = core.read_model(model=model_path)
compiled_model = core.compile_model(model, device)
print(f"Cache disabled - compile time: {time.time() - start}s")

Cache disabled - compile time: 2.488774299621582s


The actual time improvements will depend on the environment as well as the model being used but it is definitely something to consider when optimizing an application. To read more about this, see the [Model Caching docs](https://docs.openvino.ai/2024/openvino-workflow/running-inference/optimize-inference/optimizing-latency/model-caching-overview.html).

### Throughput and Latency Performance Hints
[back to top ⬆️](#Table-of-contents:)

To simplify device and pipeline configuration, OpenVINO provides high-level performance hints that automatically set the batch size and number of parallel threads for inference. The "LATENCY" performance hint optimizes for fast inference times while the "THROUGHPUT" performance hint optimizes for high overall bandwidth or FPS.

To use the "LATENCY" performance hint, add `{"PERFORMANCE_HINT": "LATENCY"}` when compiling the model as shown below. For NPU, this automatically minimizes the batch size and number of parallel streams such that all of the compute resources can focus on completing a single inference as fast as possible.

In [16]:
compiled_model = core.compile_model(model, device, {"PERFORMANCE_HINT": "LATENCY"})

To use the "THROUGHPUT" performance hint, add `{"PERFORMANCE_HINT": "THROUGHPUT"}` when compiling the model. For NPUs, this creates multiple processing streams to efficiently utilize all the execution cores and optimizes the batch size to fill the available memory.

In [17]:
compiled_model = core.compile_model(model, device, {"PERFORMANCE_HINT": "THROUGHPUT"})

## Performance Comparison with benchmark_app
[back to top ⬆️](#Table-of-contents:)


Given all the different options available when compiling a model, it may be difficult to know which settings work best for a certain application. Thankfully, OpenVINO provides `benchmark_app` - a performance benchmarking tool.

The basic syntax of `benchmark_app` is as follows:

`
benchmark_app -m PATH_TO_MODEL -d TARGET_DEVICE -hint {throughput,cumulative_throughput,latency,none}
`

where `TARGET_DEVICE` is any device shown by the `available_devices` method as well as the MULTI and AUTO devices we saw previously, and the value of hint should be one of the values between brackets.

Note that benchmark_app only requires the model path to run but both device and hint arguments will be useful to us. For more advanced usages, the tool itself has other options that can be checked by running `benchmark_app -h` or reading the [docs](https://docs.openvino.ai/2024/learn-openvino/openvino-samples/benchmark-tool.html). The following example shows us to benchmark a simple model, using a NPU with latency focus:

`benchmark_app -m {model_path} -d NPU -hint latency`

For completeness, let us list here some of the comparisons we may want to do by varying the device and hint used. Note that the actual performance may depend on the hardware used. Generally, we should expect NPU to be better than CPU.

#### NPU vs CPU with Latency Hint
[back to top ⬆️](#Table-of-contents:)

In [18]:
!benchmark_app -m {model_path} -d CPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 20.09 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / 

In [25]:
!benchmark_app -m {model_path} -d NPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] Device info:
[ INFO ] NPU
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 12.52 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / 

##### Effects of UMD Model Caching
[back to top ⬆️](#Table-of-contents:)

To see the effects of UMD Model caching, we are going to run the benchmark_app and see the difference in model read time and compilation time:

In [26]:
!benchmark_app -m {model_path} -d NPU -hint latency

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] Device info:
[ INFO ] NPU
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 10.24 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / 

As you can see from above, it takes less time read and compile the model after the first load of the model.

#### NPU vs CPU with Throughput Hint
[back to top ⬆️](#Table-of-contents:)

In [21]:
!benchmark_app -m {model_path} -d CPU -hint throughput

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 17.19 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / 

In [22]:
!benchmark_app -m {model_path} -d NPU -hint throughput

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] Device info:
[ INFO ] NPU
[ INFO ] Build ................................. 2024.1.0-14943-021fb933e38
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 12.16 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : f32 / [...] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / [1,1000]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch size: 1
[Step 6/11] Configuring input of the model
[ INFO ] Model inputs:
[ INFO ]     input.1 (node: input.1) : u8 / [N,C,H,W] / [1,3,224,224]
[ INFO ] Model outputs:
[ INFO ]     495 (node: 495) : f32 / [...] / 

## Limitations
[back to top ⬆️](#Table-of-contents:)

1. Currently, only the models with static shapes are supported on NPU.
2. If the path to the model file includes non-Unicode symbols, such as in Chinese, the model cannot be used for inference on NPU. It will return an error.
3. Running the AlexNet model with NPU may result in a drop in accuracy. At this moment, the googlenet-v4 model is recommended for classification tasks.

## Conclusion
[back to top ⬆️](#Table-of-contents:)

This tutorial demonstrates how easy it is to use NPU in OpenVINO, check its properties, and even tailor the model performance through the different performance hints. 